In [23]:
# -*- coding: utf-8 -*-
"""
Created on Sat Feb  9 01:47:27 2019

@author: avaca
"""
import pandas as pd
import numpy as np
import pickle
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.metrics import median_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer

data = pd.read_csv('transformados04.csv')
data['exit_rate_page_views'] = data['GA_exit_rate']*data['GA_page_views']
data['views_precio_metro'] = data['GA_page_views'] * data['precio_metros']
target = data['TARGET']

In [24]:
Xt = data.drop(['TARGET'], axis = 1)

yt = data['TARGET']

In [25]:
X_train, X_test, y_train, y_test = train_test_split(Xt, yt,
                                                    test_size = 0.1,
                                                    random_state = 69)

# Parámetros

In [26]:
# Número de árboles en el random forest
n_estimators = [int(x) for x in np.linspace(start = 500, stop = 1200, num = 50)]

# Número de features a considerar en cada split
max_features = ['auto', 'sqrt']

# Máximo número de niveles en el árbol
max_depth = [None, 1, 2, 3]

# Número mínimo de samples para hacer un split en un nodo
min_samples_split = [2, 5, 10]

# Número mínimo de samples para cada hoja
min_samples_leaf = [1, 2, 4]

# Método de selección para cada árbol de entrenamiento
bootstrap = [True, False]

In [31]:
param_grid = {'n_estimators': [500],
               'max_features': ['auto'],
               'max_depth': [1],
               'min_samples_split': [2,5],
               'min_samples_leaf': [2],
               'bootstrap': [True]}

In [28]:
scorer = make_scorer(median_absolute_error)

# RandomForest

In [32]:
rf = GridSearchCV(RandomForestRegressor(criterion="mse"),
                  param_grid,
                  cv = 5,
                  scoring = scorer,
                  n_jobs = -1,
                  verbose = 1)

In [33]:
rf.fit(X_train, y_train)

pred_rf = np.exp(rf.predict(X_test)) - 1
pred_rf = pred_rf.reshape(-1, 1)

df = pd.DataFrame(np.exp(y_test)-1 , index = y_test.index)
df.columns = ['TARGET']
df['pred_rf'] = pred_rf

median_absolute_error(df['TARGET'],df['pred_rf'])

Fitting 5 folds for each of 2 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   35.1s finished


26.845877129212248

In [34]:
filename = 'random_forest_gridsearchcv1.sav'
pickle.dump(rf.best_estimator_, open(filename, 'wb'))

# ExtraTreesRegressor

In [37]:
etr = GridSearchCV(ExtraTreesRegressor(criterion="mse"),
                   param_grid,
                   cv = 5,
                   scoring = scorer,
                   n_jobs = -1,
                   verbose = 1)

In [38]:
etr.fit(X_train, y_train)

pred_etr = np.exp(etr.predict(X_test)) - 1
pred_etr = pred_etr.reshape(-1, 1)

df['pred_etr'] = pred_etr

median_absolute_error(df['TARGET'],df['pred_etr'])

Fitting 5 folds for each of 2 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   13.2s finished


25.73169263002879

In [39]:
filename = 'extra_trees_gridsearchcv1.sav'
pickle.dump(etr.best_estimator_, open(filename, 'wb'))